In [ ]:
!pip install labelme tensorflow opencv-python matplotlib albumentations

In [13]:
import os
import cv2
import uuid
import time
import random

In [14]:
# COLLECT IMAGES VIA WEBCAM

img_number = 10
img_path = os.path.join('data', 'images')

cap = cv2.VideoCapture(0)

for img_num in range(img_number):
    print("Skupljam sliku {}".format(img_num))
    img_name = os.path.join(img_path, f'{str(uuid.uuid1())}.jpg')
    ret, frame = cap.read()
    cv2.imwrite(img_name, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.3)
    
cap.release()
cv2.destroyAllWindows()

Skupljam sliku 0
Skupljam sliku 1
Skupljam sliku 2
Skupljam sliku 3
Skupljam sliku 4
Skupljam sliku 5
Skupljam sliku 6
Skupljam sliku 7
Skupljam sliku 8
Skupljam sliku 9


In [15]:
import tensorflow as tf
import numpy as np
import albumentations as alb
import json
from matplotlib import pyplot as plot

In [20]:
# Annotate images

!labelme

[INFO   ] __init__:get_config:70 - Loading config file from: C:\Users\Hrvoje\.labelmerc


In [29]:
# Partition training images

dest = os.path.join('data', 'train', 'images')

for i in range(round(img_number * 0.7)):
    random_file = random.choice(os.listdir(img_path))
    source = os.path.join(img_path, random_file)
    os.replace(source, os.path.join(dest, random_file))
    
# Partition testing images

dest = os.path.join('data', 'test', 'images')

for i in range(round(img_number * 0.15)):
    random_file = random.choice(os.listdir(img_path))
    source = os.path.join(img_path, random_file)
    os.replace(source, os.path.join(dest, random_file))
    
# Partition validation images

dest = os.path.join('data', 'val', 'images')

for f in os.listdir(img_path):
    source = os.path.join(img_path, f)
    os.replace(source, os.path.join(dest, f))

In [31]:
# CREATE DATASET
# Move matching labels

for folder in ['train', 'test', 'val']:
    for file in os.listdir(os.path.join('data', folder, 'images')):
        existing_filepath = os.path.join('data', 'labels', file.split('.')[0]+'.json')
        filename = file.split('.')[0]+'.json'
        
        if os.path.exists(existing_filepath): 
            new_filepath = os.path.join('data', folder, 'labels', filename)
            os.replace(existing_filepath, new_filepath)

In [32]:
# Augment images for more samples

augmentor = alb.Compose([alb.RandomCrop(width = 450, height = 450), 
                         alb.HorizontalFlip(p = 0.5), 
                         alb.RandomBrightnessContrast(p = 0.2),
                         alb.RandomGamma(p = 0.2), 
                         alb.RGBShift(p = 0.2), 
                         alb.VerticalFlip(p = 0.5)], 
                         keypoint_params = alb.KeypointParams(format = 'xy', label_fields = ['class_labels']))

In [33]:
for partition in ['train', 'test', 'val']: 
    for image in os.listdir(os.path.join('data', partition, 'images')):
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json')
        classes = [0,0]
        coords = [0,0,0.00001,0.00001]
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)
    
            if label['shapes'][0]['label']=='LeftEye': 
                classes[0] = 1
                coords[0] = np.squeeze(label['shapes'][0]['points'])[0]
                coords[1] = np.squeeze(label['shapes'][0]['points'])[1]

            if label['shapes'][0]['label']=='RightEye':
                classes[1] = 1
                coords[2] = np.squeeze(label['shapes'][0]['points'])[0]
                coords[3] = np.squeeze(label['shapes'][0]['points'])[1]

            if len(label['shapes']) > 1:     
                if label['shapes'][1]['label'] =='LeftEye': 
                    classes[0] = 1 
                    coords[0] = np.squeeze(label['shapes'][1]['points'])[0]
                    coords[1] = np.squeeze(label['shapes'][1]['points'])[1]

                if label['shapes'][1]['label'] =='RightEye': 
                    classes[1] = 1
                    coords[2] = np.squeeze(label['shapes'][1]['points'])[0]
                    coords[3] = np.squeeze(label['shapes'][1]['points'])[1]
            
            np.divide(coords, [640,480,640,480])
                
        try: 
            for x in range(120):
                keypoints = [(coords[:2]), (coords[2:])]
                augmented = augmentor(image=img, keypoints=keypoints, class_labels=['LeftEye','RightEye'])
                
                cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image
                annotation['class'] = [0,0]
                annotation['keypoints'] = [0,0,0,0]

                if os.path.exists(label_path):
                    if len(augmented['keypoints']) > 0: 
                        for idx, cl in enumerate(augmented['class_labels']):
                            if cl == 'LeftEye': 
                                annotation['class'][0] = 1 
                                annotation['keypoints'][0] = augmented['keypoints'][idx][0]
                                annotation['keypoints'][1] = augmented['keypoints'][idx][1]
                            if cl == 'RightEye': 
                                annotation['class'][1] = 1 
                                annotation['keypoints'][2] = augmented['keypoints'][idx][0]
                                annotation['keypoints'][3] = augmented['keypoints'][idx][1]
                                
                annotation['keypoints'] = list(np.divide(annotation['keypoints'], [450,450,450,450]))


                with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)